We are given a matrix operator 
$$
T_A:\mathbb{M}^{n+1} \to \mathbb{M}^{n+1}
$$
defined as 
$$
T_A(B) = AB
$$
where $A,B$ are $n+1 \times n+1$ matrices.

We wish to see $T_A$ as an $(n+1)^2 \times (n+1)^2$ matrix acting on vectors of size $(n+1)^2$ so that Python can compute its eigenvalues. 

We'd create a change of basis map $\Psi : \mathbb{M}^{n+1} \to \mathbb{R}^{(n+1)^2}$ so that corresponding to our operator $T_A$, we have a matrix $\mathbf{A}_{(n+1)^2\times (n+1)^2}$ so that, for a matrix $B$, defining $\mathbf{b}_{(n+1)^2 \times 1} := \Psi(B)$, we have 
$$ \Psi^{-1}[\mathbf{A} \mathbf{b}] = T_A(B) $$


We order the standard basis $$\{E_{ij}\}_{0 \le i,j \le n}$$ as 
$$\{E_{00},E_{01},\dots,E_{0n},E_{10},E_{11},\dots,E_{nn} \}.$$
That is, we choose the basis to be $\{e_k\}_{k:0}^{(n+1)^2}$ where
$$e_k = E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)} $$
where $[x]$ gives the greatest integer less than $x$.

We begin by verifying that our formula indeed forms the basis.

In [82]:
import numpy as np
from numpy import floor,sqrt

In [34]:
n=2
for k in range(0,(n+1)**2):
    print([floor(k/(n+1)),k-np.floor(k/(n+1))*(n+1)])

[0.0, 0.0]
[0.0, 1.0]
[0.0, 2.0]
[1.0, 0.0]
[1.0, 1.0]
[1.0, 2.0]
[2.0, 0.0]
[2.0, 1.0]
[2.0, 2.0]


Thus, our ordering of basis we valid.

Let's use the formula we verified above to write a matrix as a vector and then back(For going back, we need to note that $E_{ij}=e_{i(n+1)+j}$.

In [142]:
def vec_rep(B): #vector representation
    n= np.shape(B)[0]-1
    b = np.zeros(((n+1)**2,1))
    for k in range(0,(n+1)**2):
        i,j =floor(k/(n+1)),k-np.floor(k/(n+1))*(n+1)
        i,j=int(i),int(j)
        b[k,0] = B[i,j]
    return np.array(b)
def mat_rep(b):
    N=np.shape(b)[0]
    assert (sqrt(N).is_integer()),'Only perfect square \
            sized 1-d arrays can be represented as matrices'
    n=sqrt(N)-1
    n=int(n)
    B = np.zeros((n+1,n+1))
    for i in range(0,n+1):
        for j in range(0,n+1):
            B[i,j]= b[i*(n+1)+j,0]
    return B
B = np.array([ \
              [1,2,3],\
              [4,5,6],\
              [7,8,9]\
             ])
b = vec_rep(B)
B = mat_rep(b)
print(b)
print(B)

[[1.]
 [2.]
 [3.]
 [4.]
 [5.]
 [6.]
 [7.]
 [8.]
 [9.]]
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


So, we can represent a matrix as an array and then go back to our matrix.
We will now use it to represent $T_A$ as a matrix. By definition of the matrix representative $\mathbf{A}$, $$T_A(e_k)=\sum_{l:0}^{(n+1)^2} \mathbf{A}_{lk} e_l \quad \text{ for } 0\le k \le (n+1)^2$$
We can solve this equation using `numpy`, but that will be too inefficient. Let's do it explicitly.

Since $e_k = E_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}$, $T_A(e_k) = AE_{\left[\frac{k}{n+1}\right],k-\left[\frac{k}{n+1}\right](n+1)}$. That is,

Letting $A_{pq}$ denote coefficients of $A$, 
$$T_A(e_k) = \sum_{i:0}^n A_{i,\left[\frac{k}{n+1}\right]}E_{i,k-\left[\frac{k}{n+1}\right](n+1)}$$
That is,
$$T_A(e_k) = \sum_{i:0}^n A_{i,\left[\frac{k}{n+1}\right]}e_{(n+1)i+k-\left[\frac{k}{n+1}\right](n+1)}$$
Thus $$
\mathbf{A}_{lk} = 
\begin{cases}
A_{i_0,\left[\frac{k}{n+1}\right]}\quad \text{ if } l=(n+1)i_0+k-\left[\frac{k}{n+1}\right](n+1) \text{ for some }i_0 \\
0, \quad \text{ otherwise}
\end{cases}
$$



So, we'd look over $k,l$ and define
$$i:=\frac{l-k+\left[\frac{k}{n+1}\right](n+1)}{n+1}.$$
If it's an integer, we will put the respective value in the matrix. A more efficient way would be to simply loop over $i$ and determine the corresponding $k,l$. I don't know how to do it though.

In [143]:
def mat_mat(A):
    n=np.shape(A)[0]-1
    AA = np.zeros(((n+1)**2,(n+1)**2))
    for k in range(0,(n+1)**2):
        for l in range(0,(n+1)**2):
            i_num = l-k+floor(k/(n+1))*(n+1)
            i_den = n+1
            i     = i_num/i_den
            if i.is_integer():
                i = int(i)
                j = int(floor(k/(n+1)))
                AA[l,k]=A[i,j]
    return AA

In [165]:
A = np.random.rand(3,3)
B = np.random.rand(3,3)
AA = mat_mat(A)
print(AA)
b = vec_rep(B)

[[0.96126544 0.         0.         0.86771501 0.         0.
  0.88768435 0.         0.        ]
 [0.         0.96126544 0.         0.         0.86771501 0.
  0.         0.88768435 0.        ]
 [0.         0.         0.96126544 0.         0.         0.86771501
  0.         0.         0.88768435]
 [0.20099336 0.         0.         0.52922141 0.         0.
  0.72361884 0.         0.        ]
 [0.         0.20099336 0.         0.         0.52922141 0.
  0.         0.72361884 0.        ]
 [0.         0.         0.20099336 0.         0.         0.52922141
  0.         0.         0.72361884]
 [0.54766312 0.         0.         0.72704187 0.         0.
  0.89309014 0.         0.        ]
 [0.         0.54766312 0.         0.         0.72704187 0.
  0.         0.89309014 0.        ]
 [0.         0.         0.54766312 0.         0.         0.72704187
  0.         0.         0.89309014]]


In [166]:
AA@b

array([[1.82037937],
       [2.04170043],
       [1.83906914],
       [1.03799071],
       [1.14913194],
       [1.08630178],
       [1.49993148],
       [1.66741981],
       [1.54036023]])

In [167]:
mat_rep(AA@b)

array([[1.82037937, 2.04170043, 1.83906914],
       [1.03799071, 1.14913194, 1.08630178],
       [1.49993148, 1.66741981, 1.54036023]])

In [168]:
A@B

array([[1.82037937, 2.04170043, 1.83906914],
       [1.03799071, 1.14913194, 1.08630178],
       [1.49993148, 1.66741981, 1.54036023]])